### Unit Test for Silver ETL Pipeline

This unit test cell validates key components of the Mailchimp Silver ETL pipeline. Specifically, the tests ensure that:

- **Reading Data from Bronze:**  
  The `read_bronze_json` function correctly reads JSON files from the Bronze container, injects the list name, and produces a non-empty DataFrame with the expected values.

- **Data Flattening and Cleaning:**  
  The `flatten_and_clean` function properly flattens nested fields such as `merge_fields`, `location`, `stats`, and converts list-type fields (`marketing_permissions`, `tags`, `_links`) into JSON strings. This test checks that the flattened DataFrame contains the correct transformed data.

- **Writing Data to Silver:**  
  The `write_to_silver` function writes the cleaned DataFrame to the correct location in the Silver container. The test verifies that the expected CSV file is created and that it contains the correct data.

These tests use Python's `unittest` framework along with the `unittest.mock` library to simulate external dependencies like the Azure Data Lake Storage (ADLS) clients (`bronze_fs` and `silver_fs`). The tests are designed to run in a Databricks environment, where the production Silver ETL code is loaded via:

```python
%run ./Mailchimp_ETL/Production_ETL/02_Silver_prod


In [0]:
#%run ./Mailchimp_ETL/Production_ETL/02_Silver_prod
# Note: All functions and globals are assumed to be defined in __main__ after the %run.

In [0]:
import unittest
from unittest.mock import patch, MagicMock
import json
import pandas as pd

class TestMailchimpSilverETL(unittest.TestCase):
    def setUp(self):
        # Fabricated realistic sample record mimicking real Mailchimp data
        self.sample_record = {
            "id": "a1b2c3d4e5f6g7h8i9j0",
            "email_address": "jane.doe@example.com",
            "unique_email_id": "abc123xyz",
            "contact_id": "contact_98765",
            "full_name": "Jane Doe",
            "web_id": 123456789,
            "email_type": "html",
            "status": "subscribed",
            "consents_to_one_to_one_messaging": True,
            "ip_signup": "192.168.1.1",
            "timestamp_signup": "2024-01-15T10:20:30+00:00",
            "ip_opt": "192.168.1.1",
            "timestamp_opt": "2024-01-16T11:21:31+00:00",
            "member_rating": 4,
            "last_changed": "2024-03-07T20:53:17+00:00",
            "language": "en",
            "vip": False,
            "email_client": "Gmail",
            "merge_fields": {
                "FNAME": "Jane",
                "LNAME": "Doe",
                "TITLE": "Director",
                "ORG": "Example Org",
                "STATE": "CA",
                "MMERGE4": "Custom Value"
            },
            "stats": {
                "avg_open_rate": 0.75,
                "avg_click_rate": 0.50
            },
            "location": {
                "latitude": 34.05,
                "longitude": -118.25,
                "gmtoff": -8,
                "dstoff": -7,
                "country_code": "US",
                "timezone": "america/los_angeles",
                "region": "CA"
            },
            "marketing_permissions": [
                {"marketing_permission_id": "perm1", "text": "Email", "enabled": True},
                {"marketing_permission_id": "perm2", "text": "Direct Mail", "enabled": False}
            ],
            "tags": [
                {"id": 101, "name": "Premium"},
                {"id": 102, "name": "Newsletter"}
            ],
            "list_id": "list_001",
            "_links": [
                {"rel": "self", "href": "https://us4.api.mailchimp.com/3.0/lists/list_001/members/a1b2c3d4e5f6g7h8i9j0"}
            ],
            "source": "Import"
        }
        # Wrap the record in a list and encode as JSON string (as stored in Bronze)
        self.sample_json = json.dumps([self.sample_record])
        self.list_name = "PremiumSubscribers"

    @patch("__main__.bronze_fs")
    def test_read_bronze_json(self, mock_bronze_fs):
        # Simulate a file path under the Bronze folder structure
        fake_path = MagicMock(is_directory=False, name=f"{BRONZE_PREFIX}/listName={self.list_name}/data_page1.json")
        mock_bronze_fs.get_paths.return_value = [fake_path]

        # Fake file client returns our sample JSON content
        fake_file_client = MagicMock()
        fake_file_client.download_file.return_value.readall.return_value = self.sample_json.encode("utf-8")
        mock_bronze_fs.get_file_client.return_value = fake_file_client

        df = read_bronze_json(self.list_name)
        self.assertFalse(df.empty, "DataFrame should not be empty")
        self.assertEqual(df.iloc[0]["email_address"], "jane.doe@example.com")
        self.assertEqual(df.iloc[0]["list_name"], self.list_name)

    def test_flatten_and_clean(self):
        # Construct a DataFrame with the sample record and add the list_name field
        df = pd.DataFrame([self.sample_record])
        df["list_name"] = self.list_name

        df_clean = flatten_and_clean(df)
        # Verify that merge_fields were flattened
        self.assertIn("merge_FNAME", df_clean.columns)
        self.assertEqual(df_clean.iloc[0]["merge_FNAME"], "Jane")
        # Verify that location fields were flattened
        self.assertIn("location_latitude", df_clean.columns)
        self.assertEqual(df_clean.iloc[0]["location_latitude"], 34.05)
        # Verify that stats fields were flattened
        self.assertIn("stats_avg_open_rate", df_clean.columns)
        self.assertEqual(df_clean.iloc[0]["stats_avg_open_rate"], 0.75)
        # Verify that marketing_permissions is converted to a JSON string
        self.assertIsInstance(df_clean.iloc[0]["marketing_permissions"], str)
        mp = json.loads(df_clean.iloc[0]["marketing_permissions"])
        self.assertEqual(mp[0]["text"], "Email")
        # Verify that tags is converted to a JSON string
        self.assertIsInstance(df_clean.iloc[0]["tags"], str)
        tags = json.loads(df_clean.iloc[0]["tags"])
        self.assertEqual(tags[0]["name"], "Premium")
        # Verify that _links is converted to a JSON string
        self.assertIsInstance(df_clean.iloc[0]["_links"], str)
        links = json.loads(df_clean.iloc[0]["_links"])
        self.assertEqual(links[0]["rel"], "self")

    @patch("__main__.silver_fs")
    def test_write_to_silver(self, mock_silver_fs):
        # Create a minimal DataFrame for testing the write function
        df = pd.DataFrame([{"email_address": "test.user@example.com", "list_name": self.list_name}])
        fake_file_client = MagicMock()
        mock_silver_fs.get_file_client.return_value = fake_file_client

        write_to_silver(df, self.list_name)

        expected_path = f"{SILVER_PREFIX}/{self.list_name}.csv"
        mock_silver_fs.get_file_client.assert_called_with(expected_path)
        fake_file_client.upload_data.assert_called_once()
        # Check that the resulting CSV content contains the test email address
        uploaded_content = fake_file_client.upload_data.call_args[0][0]
        self.assertIn("test.user@example.com", uploaded_content)

if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)
